In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

path = '/usr/local/bin/chromedriver'
website = 'https://indiarailinfo.com/trains/longestroutes'
service = Service(executable_path=path)
options = Options()
driver = webdriver.Chrome(service=service, options=options)
driver.get(website)

# Keep clicking "Next" until no more pages
while True:
    try:
        next_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//button[@class="nextbtn"]'))
        )
        next_button.click()
        time.sleep(1)  # wait for data to load
    except:
        break

from bs4 import BeautifulSoup
import pandas as pd

# Step 1: Get updated HTML from Selenium
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Step 2: Locate the container/table
table = soup.find('div', class_="srhres newbg inline alt")

# Step 3: Extract column titles (assuming they are in the 3rd div)
header_div = table.find_all('div')[2]
title = [t.text.strip() for t in header_div.find_all('div')]
print("Column titles:", title)

# Step 4: Create DataFrame with those column names
df = pd.DataFrame(columns=title)

# Step 5: Extract rows of data (from the 5th div onward)
rows = table.find_all('div')[4:]
for row in rows:
    row_data = row.find_all('div')
    individual_row_data = [data.text.strip() for data in row_data]
    
    # Only add rows that have actual data
    if individual_row_data and any(item != '' for item in individual_row_data):
        if len(individual_row_data) == len(title):  # Ensure column count matches
            df.loc[len(df)] = individual_row_data
        else:
            print("Skipping row due to column mismatch:", individual_row_data)

# Step 6: Show the final DataFrame
df


Found 7338 entries


In [13]:
# converting to the csv 
df.to_csv(r'/Users/apple/Downloads/train_route_2.csv',index=False)